# Paper no.1

## Description

This notebook contains material for the first paper.

In [11]:
# General imports.
import os, sys

# Add the source code directory in the search path.
module_path = os.path.abspath(os.path.join('../src/'))

if module_path not in sys.path:
    sys.path.append(module_path)
# _end_if_

import h5py
import json
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from time import perf_counter
from collections import defaultdict
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import (accuracy_score,
                             confusion_matrix,
                             RocCurveDisplay,
                             classification_report,
                             ConfusionMatrixDisplay,
                             balanced_accuracy_score)

from skopt import BayesSearchCV
from skopt.space import (Real,
                         Integer,
                         Categorical)

# Custom code imports.
from metal_pdb_data import MetalPdbData
from metal_auxiliaries import MetalAtom, METAL_TARGETS
from dual_output_classification import DualOutputClassifier

In [2]:
# Dictionary of class targets:
# 1) the first value corresponds to the numerical target
# 2) and the second to its number of coordinates
CLASS_TARGETS = {"LIN": (0, 2),
                 "TRI": (1, 3),
                 "TET": (2, 4),
                 "SPL": (3, 4),
                 "SQP": (4, 5),
                 "TBP": (5, 5),                 
                 "OCT": (6, 6)}

# List of classes (+ coordinates systems).
LIST_OF_CLASS = [("lin", "2_coord"),
                 ("tri", "3_coord"),
                 ("tet", "4_coord"),
                 ("spl", "4_coord"),
                 ("sqp", "5_coord"),
                 ("tbp", "5_coord"),
                 ("oct", "6_coord")]

## Data locations:

Sets the directories where the datafiles are stored. These are HDF files that include:

1. the extracted angles
2. the distance maps
3. the file-ids
4. the minimum length

In [3]:
# CSD (training) data.
# Size: (109001, 37)
csd_path = Path("../data/CSD/")

# PDB (testing) data.
# Size: (2960, 37)
pdb_path = Path("../data/PDB/")

# Output (temporary) directory.
output_path = Path("../results/")

## Extract angles and contact maps and create the input data from the '.h5' files:

Here we read all the '.h5' files and create the required data files, that will be
used from the machine learning algorithms.

In [ ]:
# Create a random number generator.
rng = np.random.default_rng(20220427)

# Set the directory you want to process:
# 1) csd_path
# 2) pdb_path
parent_dir = csd_path

# Localize the triu_indices method.
np_triu_indices = np.triu_indices

# Store the data for each class.
data_vectors = []

# Counter the instances of each class.
class_counter = defaultdict(int)

# Process all files in the path.
for f_path in sorted([*parent_dir.glob("*.h5")]):
    
    # Split the filename.
    fname_split = f_path.stem.split("_")
    
    # Class label.
    class_label = str(fname_split[-1])
    
    # Load the data file.
    with h5py.File(f_path, "r") as data_file:
        
        # Convert directly to numpy.
        metal_dist_map = np.array(data_file["dist_map"], dtype=float)
        
        # Convert directly to numpy.
        metal_angles = np.array(data_file["angles"], dtype=float)        
    # _end_with_
        
    # Process all vectors for the class.
    for arr_i, ang_i in tqdm(zip(metal_dist_map, metal_angles),
                             f" Processing data for -> Class = {class_label} "): 
        
        # Count the class.
        class_counter[class_label] += 1
        
        # Total input vector.
        xx_total = []
        
        # Size of matrix.
        L = arr_i.shape[0]

        # Get the upper triangular values.
        xx_1 = arr_i[np_triu_indices(L, k=1)]

        # Augment the upper triangular matrix.
        xx_total.extend(xx_1)
                    
        # Augment the angles vector.
        xx_total.extend(ang_i)
        
        # Add the target value.
        xx_total.append(CLASS_TARGETS[class_label][0])
        
        # Sanity check.
        if len(xx_total) == 0:
            
            # Something went wrong.
            raise RuntimeError(" Something went wrong. You need data ... ")
        # _end_if_ 
        
        # Append the complete vector.
        data_vectors.append(xx_total)
        
    # _end_for_
    
# _end_for_

# Convert the dataset to numpy array.
data = np.array(data_vectors, dtype=float)

# Shuffle the data.
rng.shuffle(data)

# Print the dimensions of the datasets.
print(f" Dataset size: {data.shape} \n")

# Create an output path.
path_out = Path("../data/csd_metal_data.h5")

# Create a HDF5 output file (write only).
with h5py.File(path_out, "w") as f_out:
    
    # Save the data to disk.
    f_out.create_dataset("data", data=data, dtype=float)
# _end_with_

# Final confirmation.
print(" Done! ")

## Make some useful plots from the data:

### 1. Plot the _pie distribution_ of the data

In [ ]:
# Set the type of data.
head_class = "CSD" # PDB

# Change the font family.
plt.rcParams['font.family'] = 'monospace'

# Number of coordinates.
num_coord_sizes = {"2-coord": class_counter["LIN"],
                   
                   "3-coord": class_counter["TRI"],
                   
                   "4-coord": class_counter["SPL"] + class_counter["TET"],
                   
                   "5-coord": class_counter["TBP"] + class_counter["SQP"],
                   
                   "6-coord": class_counter["OCT"]}

# Create a new figure.
fig, ax = plt.subplots(figsize=(12, 12))

# Select a Qualitative color mpa with enough colours.
cmap = plt.get_cmap("tab20c")

# The outer-circle (num-coords).
outer_colors = cmap([0, 4, 8, 12, 16])

# The inner-circle (geom-classes).
inner_colors = cmap([1,
                     6,
                     9, 11,
                     13, 15, 
                     18])

# Adjust the radius.
size = 0.25

# First plot the outer-circle.
ax.pie(num_coord_sizes.values(), radius=1.0,
       colors=outer_colors, wedgeprops=dict(width=size, edgecolor='w'),
       labels=num_coord_sizes.keys(), labeldistance=0.8)

# Then plot the inner-circle.
ax.pie(class_counter.values(), radius=1.0-size,
       colors=inner_colors, wedgeprops=dict(width=size, edgecolor='w'),
       labels=class_counter.keys(), labeldistance=0.8)

# Calculate the percentage of each class.
tot = sum(class_counter.values())

# Final string.
str_ff = "Percentages: \n"

for x, y in class_counter.items():
    
    pct = np.round(y / tot*100.0, 2)
    
    # str_ff += "{0:}:{1:>6}% \n".format(x, pct)
    str_ff += "{0:}: {1:5.2f}% \n".format(x, pct)
# _end_for_

# Add the legend.
ax.text(1.1, -1.0, str_ff, size=20)

# Title.
ax.set_title(f"{head_class} (metal) files", fontsize=24)

# Make the axis symmetric.
ax.axis('equal')

# Save the figure.
plt.savefig(Path(output_path/"plots"/f"{head_class}_Class_Pie.png"), bbox_inches="tight",
            dpi=300, facecolor='w', edgecolor='w', orientation='landscape')

# Change the font family.
plt.rcParams['font.family'] = 'sans'

## Load the data for ML algorithms:

TBD...

In [4]:
# Set the train/test type.
train_type = "CSD" # {CSD or PDB}
test_type = "CSD"  # {CSD or PDB}

# Complete structure.
data_set = {"Train": None, "Test": None}

# Test size ~ 10%.
test_split = 0.10

# Load the TRAIN data files.
if train_type == "CSD":
    
    # Load train datasets.
    with h5py.File(Path("../data/csd_metal_data.h5"),
                   mode="r") as hdf_file:
        
        # Load the data.
        CSD_DATA = np.array(hdf_file["data"], dtype=float)
    # _end_with_
    
    # Check the test set type.
    if test_type == "CSD":
        
        # Total number of test points.
        test_size = int(len(CSD_DATA) * test_split)

        # Split test set.
        data_set["Test"] = CSD_DATA[0:test_size, :]

        # Split train set.
        data_set["Train"] = CSD_DATA[test_size:, :]
    
    else:
        
        # Load PDB data.
        with h5py.File(Path("../data/pdb_metal_data.h5"),
                       mode="r") as hdf_file:

            # Load the data.
            PDB_DATA = np.array(hdf_file["data"], dtype=float)
        # _end_with_
        
        # Put the data together.
        data_set = {"Train": CSD_DATA, "Test": PDB_DATA}
    # _end_if_
    
else:
    
    # Load PDB data.
    with h5py.File(Path("../data/pdb_metal_data.h5"),
                   mode="r") as hdf_file:

        # Load the data.
        PDB_DATA = np.array(hdf_file["data"], dtype=float)
    # _end_with_
    
    # Total number of test points.
    test_size = int(len(PDB_DATA) * test_split)

    # Split test set.
    data_set["Test"] = PDB_DATA[0:test_size, :]

    # Split train set.
    data_set["Train"] = PDB_DATA[test_size:, :]
# _end_if_

# Split test/train data.
x_train = data_set["Train"][:, 0:-1].copy()
x_test = data_set["Test"][:, 0:-1].copy()

y_train = data_set["Train"][:, -1].copy()
y_test = data_set["Test"][:, -1].copy()

# Compute the missing values int the two sets.
train_missing = np.round(100.0 * np.sum(x_train == -1.0)/x_train.size, 3)
test_missing = np.round(100.0 * np.sum(x_test == -1.0)/x_test.size, 3)

# Print the dimensions of the datasets.
print(f"{train_type} --> {test_type}")
print(f"Dataset sizes: ")
print(f"X-train: {x_train.shape}, y-train: {y_train.shape}, Missing values: {train_missing}%")
print(f"X-test : {x_test.shape}, y-test: {y_test.shape}, Missing values: {test_missing}%")
print("Done!\n")

CSD --> CSD
Dataset sizes: 
X-train: (98101, 36), y-train: (98101,), Missing values: 0.014%
X-test : (10900, 36), y-test: (10900,), Missing values: 0.015%
Done!



## Bayesian HPO

In [ ]:
# Create the search space for the HPO.
search_space = {"mlp__activation": Categorical(['relu', 'tanh']),
                "mlp__hidden_layer_sizes": Integer(10, 1024),
                "mlp__shuffle": Categorical([True, False]),
                "mlp__alpha": Real(1.0e-4, 1.0e-1,
                                   prior="log-uniform"),
                "mlp__learning_rate_init": Real(1.0e-3, 1.0e-1,
                                                prior="log-uniform"),
                "mlp__batch_size": Integer(16, 1024)}

# Create the pipeline.
hpo_pipeline=Pipeline(steps=[('data_scaler', RobustScaler(copy=True)),
                             ('mlp', MLPClassifier(solver="adam", max_iter=200,
                                                   early_stopping=False))])

# Search across different combinations using Bayesian optimization.
hpo_search = BayesSearchCV(estimator=hpo_pipeline, search_spaces=search_space, refit=True,
                           n_iter=50, cv=5, scoring="balanced_accuracy", verbose=5, n_jobs=5,
                           return_train_score=True, random_state=911, error_score=np.nan)
# First time.
t0 = perf_counter()

# Display the model we are optimizing.
print(f"Optimizing ANN model ...\n")

# Fit the HPO search model.
hpo_search.fit(x_train, y_train)

# Final time.
tf = perf_counter()

# Display information.
print(f"HPO finished in {tf-t0:.2f} seconds.\n")

# Display best parameters.
print(f" Best score {hpo_search.best_score_}, with parameters are: \n")
for b_param, b_value in hpo_search.best_params_.items():
    print(f" {b_param} --> {b_value}")
# _end_for_

# Save the best model.
joblib.dump(hpo_search.best_estimator_,
            Path(f"../models/HPO_{train_type}_{test_type}_CV.model"),
            compress="zlib")

# Display final message.
print(" Done!")

In [ ]:
"""
 CSD->CSD
 --------
 Best score 0.9838429768249363, with parameters are: 

 mlp__activation --> tanh
 mlp__alpha --> 0.1
 mlp__batch_size --> 1024
 mlp__hidden_layer_sizes --> 1024
 mlp__learning_rate_init --> 0.0013556613475402466
 mlp__shuffle --> True

 PDB->PDB
 --------
 Best score 0.886341131565478, with parameters are: 

 mlp__activation --> tanh
 mlp__alpha --> 0.0001
 mlp__batch_size --> 762
 mlp__hidden_layer_sizes --> 1024
 mlp__learning_rate_init --> 0.001
 mlp__shuffle --> True
"""

### ML model: SciKit ANN

In [37]:
# Create a model.
'''
clf_model = Pipeline(steps=[("data-scaler", RobustScaler(copy=True)),
                            ("mlp", MLPClassifier(hidden_layer_sizes=(1024,),
                                                  shuffle=True,
                                                  batch_size=1024,
                                                  activation="relu",
                                                  solver="adam",
                                                  alpha=0.1,
                                                  learning_rate_init=0.0013556613475402466,
                                                  epsilon=1.0e-5,
                                                  max_iter=15000,
                                                  random_state=0,
                                                  verbose=0))])
'''
# Or, load an HPO model.
clf_model = joblib.load(Path(f"../models/HPO_{train_type}_{test_type}_CV.model"))
# clf_model = joblib.load(Path(f"../models/CSD_PDB_2022_07_08_10_45_49.joblib"))

In [ ]:
# timeit clf_model.predict(x_test[0].reshape(1, -1))
# 253 µs ± 8.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

## Cross-validation Score

In [ ]:
# First time.
t0 = perf_counter()

# Run the K-Fold cross-validation.
cv_scores = cross_val_score(clf_model, x_train, y_train,
                            scoring="balanced_accuracy",
                            n_jobs=5, verbose=0, cv=5)
# Final time.
tf = perf_counter()

# Display information.
print(f"Finished K-Fold cross-validation in {tf-t0:.2f} seconds.\n")

# Print the scores.
print("Cross-validation Scores: ")
for i, sc in enumerate(cv_scores, start=1):
    print(f"Fold: {i}, Score= {sc}")
# _end_for_

print(f"Mean={cv_scores.mean():.4f}, STD= {cv_scores.std():.4f}")

In [ ]:
'''
CSD->CSD
--------
Cross-validation Scores (of the HPO model): 
Fold: 1, Score= 0.9811558773388793
Fold: 2, Score= 0.9835962285170920
Fold: 3, Score= 0.9833716168956926
Fold: 4, Score= 0.9837192613894198
Fold: 5, Score= 0.9823621694776444
Mean=0.9828, STD= 0.0010

PDB->PDB
--------
Cross-validation Scores (of the HPO model): 
Fold: 1, Score= 0.8607687872184141
Fold: 2, Score= 0.8998860039715499
Fold: 3, Score= 0.8155114226375908
Fold: 4, Score= 0.9071965187420980
Fold: 5, Score= 0.8880859582182694
Mean=0.8743, STD= 0.0334
'''

## Fit the model to the data.

In [ ]:
# Display information.
print("Started fitting ANN model ...")

# First time. 
t0 = perf_counter()

# Fit the model.
# clf_model.fit(x_train, y_train)

# Second time.
t1 = perf_counter()

# Display information.
print(f"Finished fitting the training set: {x_train.shape} in {t1-t0:.2f} seconds.\n")

# Get the predictions on the test set.
y_pred = clf_model.predict(x_test)

# Third time.
t2 = perf_counter()

# Display information.
print(f"Finished predicting test set: {x_test.shape}, in {t2-t1:.2f} seconds.\n")

# Get the final report.
clf_rep = classification_report(y_test, y_pred, output_dict=True,
                                target_names=CLASS_TARGETS.keys())
# Create a large figure.
_, ax = plt.subplots(figsize=(12, 12))

# Create a Confusion matrix.
matrix = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=ax, cmap=plt.cm.GnBu,
                                                 xticks_rotation=45.0, normalize="true",
                                                 display_labels=CLASS_TARGETS.keys(),
                                                 values_format=".2f", colorbar=False)
# Setup the title.
ax.set_title("Balanced Accuracy = {0:.2f}%, F1-weighted = {1:.2f}%".format(100.0 * balanced_accuracy_score(y_test, y_pred),
                                                                           100.0 * clf_rep["weighted avg"]["f1-score"]), fontsize=18)
ax.set_xlabel("Predicted label", fontsize=16)
ax.set_ylabel("True label", fontsize=16)

# Set the custom colorbar.
matrix.im_.autoscale()
plt.colorbar(matrix.im_, fraction=0.046, pad=0.04)

# Save the figure.
plt.savefig(Path(output_path/"plots"/f"{train_type}_{test_type}_confusion_matrix.png"),
            bbox_inches="tight", dpi=300, facecolor='w', edgecolor='w',
            orientation='landscape')

# Save the classification to a json file.
with open(Path(output_path/"reports"/f"{train_type}_{test_type}_classification_report.json"),
          mode='w') as json_file:
    json.dump(clf_rep, json_file, indent=4)
# _end_with_

# Get the timestamp.
date_now = datetime.now().strftime("%Y_%m_%d_%I_%M_%S")

# Save the trained model.
joblib.dump(clf_model,
            Path(f"../models/{train_type}_{test_type}_{date_now}.joblib"),
            compress=4)

# Get the loss-curve.
loss_curve = clf_model["mlp"].loss_curve_

# Plot the figure.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(loss_curve)
ax.set_title(f"Multi Layer Perceptron Classifier")
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
plt.grid(True)

# Save the figure.
plt.savefig(Path(output_path/"plots"/f"{train_type}_{test_type}_loss_curve.png"),
            bbox_inches="tight", facecolor='w', edgecolor='w',
            dpi=300, orientation="landscape")

In [ ]:
cm_= pd.DataFrame(confusion_matrix(y_test, y_pred))

FP = cm_.sum(axis=0) - np.diag(cm_)  
FN = cm_.sum(axis=1) - np.diag(cm_)
TP = np.diag(cm_)
TN = cm_.values.sum() - (FP + FN + TP)

# Support (number of instances of each class)
SUP = cm_.sum(axis=1)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

# F1-Score
F1S = 2.0 * (PPV * TPR) / (PPV + TPR)

df = pd.DataFrame({"F1S": F1S, "TPR": TPR, "TNR": TNR, "PPV": PPV,
                   "NPV": NPV, "FPR": FPR, "FNR": FNR, "FDR": FDR})

df.index = list(CLASS_TARGETS.keys())

df.to_csv(Path(f"../results/reports/{train_type}_{test_type}_per_class.csv"))

print(df.T)

## Probability bar-plots

In [ ]:
# Get the predicted probabilities on the test set.
y_probs = clf_model.predict_proba(x_test)

# Store the probabilities per class.
probs_per_class = defaultdict(list)

# Group result by true class.
for v, u in zip(y_test, y_probs):
    probs_per_class[v].append(u)
# _end_for_

# Get the class labels.
class_label = list(CLASS_TARGETS.keys())

# Create a new figure.
fig, ax = plt.subplots(nrows=2, ncols=4,
                       figsize=(18, 12))

# Delete the last plot.
fig.delaxes(ax[1][3])

# Put all the bar-plots together.
for c_num, c_label in enumerate(probs_per_class, start=0):
        
    # Enable the subplot.
    plt.subplot(2, 4, c_num+1)
    
    # Get the proabilities for each predicted class.
    prob_c = np.sum(probs_per_class[c_num], axis=0)/len(probs_per_class[c_num])
    
    # Get the indexes of the sorted vector (descending order).
    descending_order = np.argsort(prob_c)[::-1]
    
    # Initial assignments of bar entries.
    heights = prob_c[descending_order]
    
    # Exclude the zero entries.
    not_equal_zero = heights!=0.0
    
    # Compute the entropy:
    En = -np.sum(heights[not_equal_zero]*np.log(heights[not_equal_zero]))
    
    # Get the relevant colours.
    colors_, labels_ = [], []
    for k in descending_order:
        colors_.append(plt.cm.tab20.colors[k])
        labels_.append(class_label[k])
    # _end_for_
        
    # Plot the normalized probabilites.
    bars = plt.bar(x=labels_, height=heights, color=colors_)
    
    # Access the bar attributes to place
    # the text in the appropriate location.
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x(), yval + .002,
                 f"{np.round(yval, 3):.3f}",
                 fontsize='x-small')
    # _end_for_
    
    # Setup grid and more.
    plt.grid()
    plt.ylim([0.0, 1.0])

    plt.ylabel("Probability", fontsize=12)
    plt.xlabel("Predicted class", fontsize=12)
    
    plt.title(f"True class: {class_label[c_num]}, Entropy {En:.4f}", fontsize=12)
# _end_for_

# Set the spacing between subplots.
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Save the figure.
plt.savefig(Path(output_path/"plots"/f"{train_type}_{test_type}_probablities_entropy.png"),
            bbox_inches="tight", dpi=300, facecolor='w', edgecolor='w',
            orientation='landscape');

### End of notebook